# Import & data

In [2]:
import arcpy, os
import arcpy
from osgeo import gdal

arcpy.env.addOutputsToMap = False   # Prevents many small grid files to show on map
arcpy.env.overwriteOutput = True

In [41]:
# Directory option 1: Use project gdb
# Directory option 2: use project directory
# Directory option 3: Custom directory
Dir = 2  # [1, 2 or 3] (Default is 1)

if Dir == 1:
    p = arcpy.mp.ArcGISProject("CURRENT")
    directory = p.defaultGeodatabase   
elif Dir == 2:
    directory = os.path.dirname( arcpy.mp.ArcGISProject("CURRENT").filePath )
elif Dir == 3:
    directory = r"H:\DATA\Service\Wietse\GIS\DEM"

arcpy.env.workspace = directory

In [42]:
Input_ArpPy = r"H:\Team_Kennis_OSA\Hydraulische toetsing\Basisbestanden Zomer\Input_ArpPy"
arcpy.env.workspace = Input_ArpPy
feature_classes = arcpy.ListFeatureClasses()

#Import all symbiologies used in this project
lyr_BUFF = os.path.join(Input_ArpPy, "Afvoergebiedmask.lyr")
lyr_OPST = os.path.join(Input_ArpPy, "Opstuwing kunstwerken.lyr")
lyr_SE = os.path.join(Input_ArpPy, "Guess_start_end_with_pg_hydro_sobek_id.lyr")
lyr_WL = os.path.join(Input_ArpPy, "waterniveau_stationaire_afvoer_sobek_droogleggingseis.lyr")

# Iterate through the list of feature classes
for fc in feature_classes:
    if "Basis_Hydraulische_Toetsing_Hydro_PG_SOBEK_with_Dis_ZOMER" in fc:
    # Import the feature class
        arcpy.FeatureClassToFeatureClass_conversion(fc, directory, str(fc))
        lyr_DIS = os.path.join(Input_ArpPy, "Basis_Hydraulische_Toetsing_Hydro_PG_SOBEK_with_Dis_ZOMER.lyr")
        arcpy.env.addOutputsToMap = True
        arcpy.ApplySymbologyFromLayer_management(os.path.join(directory, str(fc)),lyr_DIS)
        arcpy.env.addOutputsToMap = False
    
    else:
        arcpy.FeatureClassToFeatureClass_conversion(fc, directory, str(fc))                                      
    
arcpy.env.workspace = directory

In [43]:
def add_and_calculate_fields(fc, fields):
    """
    A function that adds and calculates multiple fields in a feature class
    
    Parameters:
    fc (str): The feature class to which the fields will be added
    fields (list): A list of fields to be added. Each field should be a list with the following elements:
                    [field_name, field_type, field_precision, field_scale, field_length, field_alias]
    expressions (list): A list of expressions to calculate the fields with
    """
    # Loop through the fields and add them
    for field in fields:
        arcpy.AddField_management(fc, field[0], field[1], field[2], field[3], field[4], field[5])


# 1 Bepalen bodem voor stroomsnelheid

In [25]:
#Import layers
Input_Arcpy_Eenmalig = r"H:\Team_Kennis_OSA\Hydraulische toetsing\Basisbestanden Zomer\Input_ArpPy\Eenmalige_bestanden"
Bodemkaart_HDSR = os.path.join(Input_Arcpy_Eenmalig,"Bodemkaart_HDSR.shp")

#-----------------
arcpy.AddField_management(Bodemkaart_HDSR, "vel_crit", "DOUBLE")

def calculate_value(field_value):
    if "Stedelijk" in field_value or "stedelijk" in field_value:
        return 0.3
    elif "Zand" in field_value or "zand" in field_value:
        return 0.3
    elif "Veen" in field_value or "veen" in field_value:
        return 0.3001
    elif "Klei" in field_value or "klei" in field_value:
        return 0.6
    elif "Zavel" in field_value or "zavel" in field_value:
        return 0.6
    elif "Water" in field_value or "water" in field_value:
        return 0.3
    elif "Moeras" in field_value or "moeras" in field_value:
        return 0.3001
    elif "Overig" in field_value or "overig" in field_value:
        return 0.3001
    else:
        return 0.9

# Calculate the values of the calculation_field using the calculate_value function
with arcpy.da.UpdateCursor(Bodemkaart_HDSR, ["GRONDSOORT", "vel_crit"]) as cursor:
    for row in cursor:
        row[1] = calculate_value(row[0])
        cursor.updateRow(row)


In [ ]:
def join_table_to_shapefile(input_shapefile, input_table, output_shapefile):
    # Set join fields
    join_field_shapefile = "ID"
    join_field_table = "LOCATION"
    fieldList = ["ID", "LOCATION", "VALUE"]
    
    NewFieldName = input_table.replace(".DBF","")
    # Save output shapefile
    arcpy.CopyFeatures_management(input_shapefile, output_shapefile)

    # Join table to shapefile
    arcpy.JoinField_management(output_shapefile, join_field_shapefile, input_table, join_field_table, fieldList)
    arcpy.AddField_management(output_shapefile, NewFieldName, "DOUBLE")
    arcpy.CalculateField_management(output_shapefile, NewFieldName, "!VALUE!", "PYTHON3")

    # Set fields to delete
    fields_to_delete = ["NAME", "TYPE", "PARENTID", "USERID","ID_FROM", "ID_TO","VALUE"]

    # Delete fields from shapefile
    arcpy.DeleteField_management(output_shapefile, fields_to_delete)


In [26]:
# Set input shapefile and table
input_shapefile = os.path.join(Input_Arcpy_Eenmalig,"RchSegments.shp")
input_table = os.path.join(Input_Arcpy_Eenmalig,"DIS.DBF")

##%----------------------------------------------------

# Set join fields
join_field_shapefile = "ID"
join_field_table = "LOCATION"
fieldList = ["ID", "LOCATION", "VALUE"]


# Set output shapefile
sobek_reach_dis =  os.path.join(Input_Arcpy_Eenmalig,"sobek_reach_dis.shp")

# Save output shapefile
arcpy.CopyFeatures_management(input_shapefile, sobek_reach_dis)

# Join table to shapefile
arcpy.JoinField_management(sobek_reach_dis, join_field_shapefile, input_table, join_field_table, fieldList)
arcpy.AddField_management(sobek_reach_dis, "DIS", "DOUBLE")
arcpy.CalculateField_management(sobek_reach_dis, "DIS", "!VALUE!", "PYTHON3")

# Set fields to delete
fields_to_delete = ["NAME", "TYPE", "PARENTID", "USERID","ID_FROM", "ID_TO","VALUE"]

# Delete fields from shapefile
arcpy.DeleteField_management(sobek_reach_dis, fields_to_delete)


<Result 'H:\\Team_Kennis_OSA\\Hydraulische toetsing\\Basisbestanden Zomer\\Input_ArpPy\\Eenmalige_bestanden\\sobek_reach_dis.shp'>

In [27]:
# Set output line layer
sobek_reach_dis_Vcrit = os.path.join(Input_Arcpy_Eenmalig, "sobek_reach_dis_Vcrit.shp")

# Set join type and match option
join_type = "JOIN_ONE_TO_MANY"
match_option = "INTERSECT"

# Set field mapping
field_mappings = arcpy.FieldMappings()
field_mappings.addTable(sobek_reach_dis)
field_mappings.addTable(Bodemkaart_HDSR)

# Perform spatial join
arcpy.SpatialJoin_analysis(sobek_reach_dis, Bodemkaart_HDSR, sobek_reach_dis_Vcrit, join_type, "KEEP_ALL", field_mappings, match_option)

fields_to_delete = ["Join_Count", "TARGET_FID", "JOIN_FID", "OBJECTID_1", "OBJECTID", "BODEMTYPE", "GRONDSOORT", "SUBGROEP", "OPBOUW", "KALK", "PROFIEL", "SHAPE_AREA", "SHAPE_LEN"]


# Delete fields from shapefile
arcpy.DeleteField_management(sobek_reach_dis_Vcrit, fields_to_delete)

# Print message when spatial join is complete
print("Spatial join of polygons and lines is complete!")


Spatial join of polygons and lines is complete!


In [12]:
import os
import arcpy

# Define input layer and table
layer = os.path.join(Input_Arcpy_Eenmalig,"RchSegments.shp")
table = os.path.join(Input_Arcpy_Eenmalig,"DIS.DBF")

# Define join fields
joinField1 = "ID"
joinField2 = "Bodemkaart_HDSR"

# Define output shapefile
outputShapefile = "sobek_reach_dis.shp"

# Perform join
arcpy.JoinField_management(layer, joinField1, table, joinField2)

# Define field mappings to only keep selected fields and rename one field
fieldMappings = arcpy.FieldMappings()
fieldMappings.addTable(layer)
fieldMappings.addFieldMap(fieldMappings.findFieldMapIndex(joinField1))
fieldMappings.addFieldMap(fieldMappings.findFieldMapIndex(joinField2))
keepFields = ["VALUE", "ID", "LOCATION"]
for field in fieldMappings.fields:
    if field.name == "VALUE":
        field.aliasName = "DIS"
    elif field.name not in keepFields:
        fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name))

# Copy features and save as new shapefile
arcpy.CopyFeatures_management(layer, outputShapefile, "", "", fieldMappings)



ValueError: FieldMappings: AddFieldMap input not field map object

In [61]:
SOBEK_Reach_segmnents_Discharge = os.path.join(Input_Arcpy_Eenmalig,"Bodemkaart_HDSR.shp")

# Process: Spatial Join
arcpy.SpatialJoin_analysis(SOBEK_Reach_segmnents_Discharge, Bodemkaart_HDSR, "test.shp")

# 2A Drooglegging bepalen (eenmalig)
- jjkk

In [ ]:
import arcpy


# Local variables:
Afwateringseenheden_2020_05_13 = "Afwateringseenheden_2020_05_13"
Afwateringseenheden_2020_05_13__2_ = Afwateringseenheden_2020_05_13



# Process: Zonal Statistics as Table
arcpy.gp.ZonalStatisticsAsTable_sa(Afwateringseenheden_2020_05_13, "CODE", AHN_buffer_mm_int, drooglegging_per_afwateringseenheid, "DATA", "MEDIAN")

# Process: Add Field
arcpy.AddField_management(drooglegging_per_afwateringseenheid, "dl_msl", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")

# Process: Calculate Field
arcpy.CalculateField_management(drooglegging_per_afwateringseenheid__2_, "dl_msl", "[MEDIAN] / 1000", "VB", "")

# Process: Add Join
arcpy.AddJoin_management(Afwateringseenheden_2020_05_13, "CODE", drooglegging_per_afwateringseenheid__3_, "CODE", "KEEP_ALL")

# 2B Droogleggingseis toetsen per punt (eenmalig)

In [ ]:
# Local variables:

# Process: Spatial Join
arcpy.SpatialJoin_analysis(waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922, v20210311_afwateringseenheden_drooglegging, waterniveau_stationaire_afvoer_sobek_droogleggingseis_2021109_shp, "JOIN_ONE_TO_ONE", "KEEP_ALL", "FID_1 \"FID_1\" true true false 9 Long 0 9 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,FID_1,-1,-1;ID \"ID\" true true false 18 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,ID,-1,-1;NAME \"NAME\" true true false 32 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,NAME,-1,-1;TYPE \"TYPE\" true true false 52 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,TYPE,-1,-1;Field1 \"Field1\" true true false 254 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,Field1,-1,-1;SOBEK_Hist \"SOBEK_Hist\" true true false 254 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,SOBEK_Hist,-1,-1;waterlevel \"waterlevel\" true true false 13 Float 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,waterlevel,-1,-1;FID_2 \"FID_2\" true true false 9 Long 0 9 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,FID_2,-1,-1;CODE \"CODE\" true true false 24 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,CODE,-1,-1;NAAM \"NAAM\" true true false 50 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,NAAM,-1,-1;OSMOMSCH \"OSMOMSCH\" true true false 60 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,OSMOMSCH,-1,-1;ONDPEILGEB \"ONDPEILGEB\" true true false 24 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,ONDPEILGEB,-1,-1;AFWATERING \"AFWATERING\" true true false 24 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,AFWATERING,-1,-1;ONDAFVOGEB \"ONDAFVOGEB\" true true false 24 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,ONDAFVOGEB,-1,-1;OPPVLK \"OPPVLK\" true true false 19 Double 8 18 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,OPPVLK,-1,-1;VASTPEIL_1 \"VASTPEIL_1\" true true false 254 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,VASTPEIL_1,-1,-1;WINTERPE_1 \"WINTERPE_1\" true true false 254 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,WINTERPE_1,-1,-1;ZOMERPEI_1 \"ZOMERPEI_1\" true true false 254 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,ZOMERPEI_1,-1,-1;BOVENPEI_1 \"BOVENPEI_1\" true true false 254 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,BOVENPEI_1,-1,-1;ONDERPEI_1 \"ONDERPEI_1\" true true false 254 Text 0 0 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,ONDERPEI_1,-1,-1;DEF_W_PEIL \"DEF_W_PEIL\" true true false 16 Double 6 15 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,DEF_W_PEIL,-1,-1;DEF_Z_PEIL \"DEF_Z_PEIL\" true true false 16 Double 6 15 ,First,#,waterniveau_stationaire_afvoer_sobek_droogleggingseis_20210922,DEF_Z_PEIL,-1,-1;CODE_1 \"CODE_1\" true true false 50 Text 0 0 ,First,#,20210311_afwateringseenheden_drooglegging,CODE,-1,-1;NAAM_1 \"NAAM_1\" true true false 250 Text 0 0 ,First,#,20210311_afwateringseenheden_drooglegging,NAAM,-1,-1;COUNT \"COUNT\" true true false 10 Long 0 10 ,First,#,20210311_afwateringseenheden_drooglegging,COUNT,-1,-1;AREA \"AREA\" true true false 19 Double 0 0 ,First,#,20210311_afwateringseenheden_drooglegging,AREA,-1,-1;MEDIAN \"MEDIAN\" true true false 10 Long 0 10 ,First,#,20210311_afwateringseenheden_drooglegging,MEDIAN,-1,-1;dl_msl \"dl_msl\" true true false 19 Double 0 0 ,First,#,20210311_afwateringseenheden_drooglegging,dl_msl,-1,-1;dl_abs \"dl_abs\" true true false 19 Double 0 0 ,First,#,20210311_afwateringseenheden_drooglegging,dl_abs,-1,-1;awlr_abs_w \"awlr_abs_w\" true true false 19 Double 0 0 ,First,#,20210311_afwateringseenheden_drooglegging,awlr_abs_w,-1,-1", "WITHIN", "0 Meters", "")

# Define the feature class
fc_2A = "example_fc"

# Define the field names and properties
fields_2A = [["awlr_msl", "FLOAT", "", "", "", "", "NULLABLE", "NON_REQUIRED"],
          ["ruimte_ver", "FLOAT", "", "", "", "", "NULLABLE", "NON_REQUIRED"]]



# Define the expressions for calculating the fields
expressions = ["awlr_msl + 1", "!Field2! * 2", "'Text'"]

# Call the function to add and calculate the fields
add_and_calculate_fields(fc, fields, expressions)

add_fields(fc_2A, fields_2A)

# Process: Calculate Field
arcpy.CalculateField_management(waterniveau_stationaire_afvoer_sobek_droogleggingseis_2021109_shp, "dl_abs", "[dl_msl] - [DEF_Z_PEIL]", "VB", "")

# Process: Calculate Field (2)
arcpy.CalculateField_management(waterniveau_stationaire_afvoer_sobek_droogleggingseis_2021109_shp__5_, "awlr_abs_z", "[dl_abs] / 2", "VB", "")

# Process: Calculate Field (4)
arcpy.CalculateField_management(waterniveau_stationaire_afvoer_sobek_droogleggingseis_2021109_shp__3_, "awlr_msl", "[DEF_Z_PEIL] + [awlr_abs_z]", "VB", "")

# Process: Calculate Field (3)
arcpy.CalculateField_management(waterniveau_stationaire_afvoer_sobek_droogleggingseis_2021109_shp__8_, "ruimte_ver", "[awlr_msl] - [waterlevel]", "VB", "")



In [ ]:
directory

# 1 Bepalen bodem voor stroomsnelheid

# 3A Exporteren data per afvoergebied

In [ ]:
# Define the string attribute value to select
Afvoergebied = "Zegveld"


#-----------------------------------------------------------------------------------------------------------------------------------------
arcpy.env.addOutputsToMap = False
selection = str(""" "NAAM" = '""" + Afvoergebied + "'")
# Set the input shapefile
input_shapefile = os.path.join(directory, "BR_Afvoergebieden.shp")

# Create a feature layer from the input shapefile
arcpy.MakeFeatureLayer_management(input_shapefile, "input_shapefile_layer")

# Select features based on the string attribute
arcpy.SelectLayerByAttribute_management("input_shapefile_layer", "NEW_SELECTION", selection)

# Save the selected features to a new feature class
arcpy.CopyFeatures_management("input_shapefile_layer", directory + "/Afvoergebied_"+ Afvoergebied + ".shp")

# Set the condition for the shapefiles to be added to the list
conditions = ["Bridges_Culverts_Head", "Guess_start_end", "waterniveau_stationaire_afvoer"]
conditions_rec = conditions
# Create an empty list to store the shapefiles
shapefiles_list = []

# Loop through the shapefiles in the workspace
for shapefile in arcpy.ListFeatureClasses(feature_type="Point"):
    for condition in conditions:
        if condition in shapefile:
            shapefiles_list.append(os.path.join(directory, shapefile))
            break

# Set the input polygon shapefile
clip = "Afvoergebied_" + Afvoergebied + ".shp"

# Run the Buffer_analysis function
arcpy.Buffer_analysis(directory + "/Afvoergebied_"+ Afvoergebied + ".shp", directory + "buffer_" + Afvoergebied + ".shp", 500, line_side = "OUTSIDE_ONLY")

# Loop through the input shapefiles and clip them with the polygon shapefile
for shapefiles_list, conditions in zip(shapefiles_list, conditions):
    output_shapefile = os.path.join(directory, conditions + "_" +Afvoergebied +  ".shp")
    arcpy.Clip_analysis(shapefiles_list, os.path.join(directory, clip), output_shapefile)

    
arcpy.env.addOutputsToMap = True

arcpy.ApplySymbologyFromLayer_management(directory + "buffer_" + Afvoergebied + ".shp",lyr_BUFF)
arcpy.ApplySymbologyFromLayer_management(os.path.join(directory, conditions_rec[0] + "_" + Afvoergebied),lyr_OPST)
arcpy.ApplySymbologyFromLayer_management(os.path.join(directory, conditions_rec[1] + "_" + Afvoergebied),lyr_SE)
arcpy.ApplySymbologyFromLayer_management(os.path.join(directory, conditions_rec[2] + "_" + Afvoergebied),lyr_WL)

selection_start = str(""" "direction" = '""" + "start" + "'")
selection_end = str(""" "direction" = '""" + "end" + "'")

arcpy.env.addOutputsToMap = False
arcpy.SelectLayerByAttribute_management(os.path.join(directory, conditions_rec[1] + "_" + Afvoergebied), "NEW_SELECTION", selection_start)
arcpy.CopyFeatures_management(os.path.join(directory, conditions_rec[1] + "_" + Afvoergebied), directory + "/Start_"+ Afvoergebied + ".shp")

arcpy.SelectLayerByAttribute_management(os.path.join(directory, conditions_rec[1] + "_" + Afvoergebied), "NEW_SELECTION", selection_end)
arcpy.CopyFeatures_management(os.path.join(directory, conditions_rec[1] + "_" + Afvoergebied), directory + "/End_"+ Afvoergebied + ".shp")

arcpy.env.addOutputsToMap = True
arcpy.ApplySymbologyFromLayer_management(directory + "/Start_"+ Afvoergebied + ".shp",lyr_SE)
arcpy.ApplySymbologyFromLayer_management(directory + "/End_"+ Afvoergebied + ".shp",lyr_SE)



In [ ]:
network = os.path.join(directory,"Basis_Hydraulische_Toetsing_Hydro_PG_SOBEK_with_Dis_ZOMER.shp")
start_points = directory + "/Start_"+ Afvoergebied + ".shp"
end_points =   directory + "/End_"+ Afvoergebied + ".shp"
output_route_layer = directory + "/Routes_"+ Afvoergebied + ".shp"

# Create a route layer
arcpy.na.MakeRouteLayer(network, output_route_layer)

# Add start and end points as stops to the route layer
arcpy.na.AddLocations(output_route_layer, "Stops", start_points)
arcpy.na.AddLocations(output_route_layer, "Stops", end_points)

# Solve the route layer to calculate the route between start and end points
arcpy.na.Solve(output_route_layer)

# 3B Filter relevante routes per afvoergebied

In [ ]:
arcpy.Save

In [ ]:
# Script arguments
Peilgebieden = arcpy.GetParameterAsText(0)
if Peilgebieden == '#' or not Peilgebieden:
    Peilgebieden = "Achtergrondlagen\\BR Peilgebieden" # provide a default value if unspecified

Sifon = arcpy.GetParameterAsText(1)
if Sifon == '#' or not Sifon:
    Sifon = "Sifon_Bijleveld" # provide a default value if unspecified

Routes_joined_to_starts = arcpy.GetParameterAsText(2)
if Routes_joined_to_starts == '#' or not Routes_joined_to_starts:
    Routes_joined_to_starts = "Export_Output" # provide a default value if unspecified

Afvoergebied = arcpy.GetParameterAsText(3)
if Afvoergebied == '#' or not Afvoergebied:
    Afvoergebied = "Afvoergebied_Bijlenveld" # provide a default value if unspecified

OD_Matrix_join_select = arcpy.GetParameterAsText(4)
if OD_Matrix_join_select == '#' or not OD_Matrix_join_select:
    OD_Matrix_join_select = "Export_Output" # provide a default value if unspecified

# Local variables:
Join_Output_2 = Routes_joined_to_starts
Join_Output_3 = Join_Output_2
Join_Output_3__2_ = Join_Output_3
Join_Output_3__5_ = Join_Output_3__2_
Join_Output_3__3_ = Join_Output_3__5_

# Process: Select Layer By Attribute
arcpy.SelectLayerByAttribute_management(Routes_joined_to_starts, "NEW_SELECTION", "Count_ >=4")

# Process: Select Layer By Location (3)
arcpy.SelectLayerByLocation_management(Join_Output_2, "WITHIN", Afvoergebied, "0 Meters", "ADD_TO_SELECTION", "INVERT")

# Process: Delete Features
arcpy.DeleteFeatures_management(Join_Output_3)

# Process: Select Layer By Location
arcpy.SelectLayerByLocation_management(Join_Output_3__2_, "WITHIN", Peilgebieden, "", "NEW_SELECTION", "INVERT")

# Process: Select Layer By Location (2)
arcpy.SelectLayerByLocation_management(Join_Output_3__5_, "INTERSECT", Sifon, "1 Meters", "REMOVE_FROM_SELECTION", "NOT_INVERT")

# Process: Delete Features (2)
arcpy.DeleteFeatures_management(Join_Output_3__3_)


